<a href="https://colab.research.google.com/github/GustavoRafael/DeepLearningProjects/blob/main/rainDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pathlib import Path
import numpy as np
import joblib
from keras.preprocessing import image
from keras.applications import vgg16

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

# adjust image size
from PIL import Image
from keras.models import model_from_json
# data source 
#https://www.kaggle.com/somesh24/multiclass-images-for-weather-classification

In [ ]:
# Path to folders with training data
rain_path = Path("/rain")
training_data_path = Path("/content/training_data")

image_count = len(list(not_rain_path.glob('*.jpg')))
print(image_count)

17


In [ ]:
images = []
labels = []

file_name = ''

# Load all the not_rain images
for img in training_data_path.glob("*.jpg"):
    file_name = img.stem[:4]
    
    # Load the image from disk
    img = image.load_img(img, target_size=(64, 64))

    # Convert the image to a numpy array
    image_array = image.img_to_array(img)

    # Add the image to the list of images
    images.append(image_array)

    # For each 'not dog' image, the expected value should be 0
    if file_name.lower() == 'rain':
      labels.append(1)
    else:
      labels.append(0)


In [ ]:
print(len(labels))
print(labels.count(0), labels.count(1))
print(len(images[0]))


186
100 86
64


In [ ]:
# Create a single numpy array with all the images we loaded
x_train = np.array(images)

# Also convert the labels to a numpy array
y_train = np.array(labels)


In [ ]:

# Normalize image data to 0-to-1 range
x_train = vgg16.preprocess_input(x_train)

# Load a pre-trained neural network to use as a feature extractor
pretrained_nn = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Extract features for each image (all in one pass)
features_x = pretrained_nn.predict(x_train)

# Save the array of extracted features to a file
joblib.dump(features_x, "x_train.dat")

# Save the matching array of expected values to a file
joblib.dump(y_train, "y_train.dat")

['y_train.dat']

In [ ]:
# Load data set
x_train = joblib.load("x_train.dat")
y_train = joblib.load("y_train.dat")


In [ ]:
# Create a model and add layers
model = Sequential()

model.add(Flatten(input_shape=x_train.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
# Compile the model
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=['accuracy']
)

# Train the model
model.fit(
    x_train,
    y_train,
    epochs=10,
    shuffle=True
)


Epoch 1/10
6/6 [==============================] - 1s 7ms/step - loss: 4.6682 - accuracy: 0.6268
Epoch 2/10
6/6 [==============================] - 0s 6ms/step - loss: 0.2186 - accuracy: 0.9724
Epoch 3/10
6/6 [==============================] - 0s 7ms/step - loss: 0.2669 - accuracy: 0.9772
Epoch 4/10
6/6 [==============================] - 0s 6ms/step - loss: 0.3510 - accuracy: 0.9775
Epoch 5/10
6/6 [==============================] - 0s 6ms/step - loss: 0.1616 - accuracy: 0.9883
Epoch 6/10
6/6 [==============================] - 0s 6ms/step - loss: 0.0092 - accuracy: 0.9950
Epoch 7/10
6/6 [==============================] - 0s 6ms/step - loss: 3.4098e-04 - accuracy: 1.0000
Epoch 8/10
6/6 [==============================] - 0s 6ms/step - loss: 0.0611 - accuracy: 0.9750
Epoch 9/10
6/6 [==============================] - 0s 6ms/step - loss: 0.0118 - accuracy: 0.9877
Epoch 10/10
6/6 [==============================] - 0s 6ms/step - loss: 3.4303e-07 - accuracy: 1.0000


In [ ]:
# Save neural network structure
model_structure = model.to_json()
f = Path("model_structure.json")
f.write_text(model_structure)

# Save neural network's trained weights
model.save_weights("model_weights.h5")


In [ ]:
# Load the json file that contains the model's structure
f = Path("model_structure.json")
model_structure = f.read_text()

# Recreate the Keras model object from the json data
model = model_from_json(model_structure)

# Re-load the model's trained weights
model.load_weights("model_weights.h5")

In [ ]:
# evaluating testing data
imagess =[]

# Path to folders with testing data
testing_path = Path("/content/test_data")

# Load all the not_rain images
for index, img in enumerate(testing_path.glob("*.jpg"), start=1):
  
  #image type
  image_type = img.stem[:4]

  # Load an image file to test, resizing it to 64x64 pixels (as required by this model)
  img = image.load_img(img, target_size=(64, 64))

  # Convert the image to a numpy array
  image_array = image.img_to_array(img)

  # Add a forth dimension to the image (since Keras expects a bunch of images, not a single image)
  images = np.expand_dims(image_array, axis=0)

  # Add the image to the list of images
  #imagess.append(image_array )

  # Normalize the data
  images = vgg16.preprocess_input(np.array(images))

  # Use the pre-trained neural network to extract features from our test image (the same way we did to train the model)
  feature_extraction_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
  features = feature_extraction_model.predict(images)

  # Given the extracted features, make a final prediction using our own model
  results = model.predict(features)
  #for ind in range(len(imagess)):
  # Since we are only testing one image with possible class, we only need to check the first result's first element
  single_result = results[0][0]
  
  # Print the result
  print("Image of {}: Likelihood that this image contains rain: {}%".format(image_type.lower(), int(single_result * 100)))

Image of rain: Likelihood that this image contains rain: 100%
Image of rain: Likelihood that this image contains rain: 100%
Image of sunn: Likelihood that this image contains rain: 0%
Image of clou: Likelihood that this image contains rain: 0%
Image of rain: Likelihood that this image contains rain: 100%
Image of sunn: Likelihood that this image contains rain: 0%
Image of rain: Likelihood that this image contains rain: 100%
Image of clou: Likelihood that this image contains rain: 0%
Image of sunn: Likelihood that this image contains rain: 0%
Image of sunn: Likelihood that this image contains rain: 0%
Image of rain: Likelihood that this image contains rain: 100%
Image of sunn: Likelihood that this image contains rain: 0%
Image of clou: Likelihood that this image contains rain: 0%
Image of rain: Likelihood that this image contains rain: 100%
Image of sunn: Likelihood that this image contains rain: 0%
Image of sunn: Likelihood that this image contains rain: 0%
